In [11]:
import csv

CSV_FILE = 'expenses.csv'

# Function to load transactions from the CSV file
def load_transactions():
    transactions = []
    try:
        with open(CSV_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                transactions.append({
                    "payer": row["payer"],
                    "amount": float(row["amount"]),
                    "description": row["description"],
                    "participants": row["participants"].split(',')
                })
    except FileNotFoundError:
        pass
    return transactions

# Function to save transactions to the CSV file
def save_transactions(transactions):
    with open(CSV_FILE, mode='w', newline='') as file:
        fieldnames = ["payer", "amount", "description", "participants"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for transaction in transactions:
            writer.writerow({
                "payer": transaction["payer"],
                "amount": transaction["amount"],
                "description": transaction["description"],
                "participants": ','.join(transaction["participants"])
            })


In [12]:
transactions = load_transactions()

def add_transaction(payer, amount, description, participants):
    transactions.append({
        "payer": payer,
        "amount": amount,
        "description": description,
        "participants": participants
    })
    save_transactions(transactions)

def edit_transaction(index, payer, amount, description, participants):
    transactions[index] = {
        "payer": payer,
        "amount": amount,
        "description": description,
        "participants": participants
    }
    save_transactions(transactions)

def delete_transaction(index):
    transactions.pop(index)
    save_transactions(transactions)


In [13]:
def calculate_balances():
    balances = {}
    for transaction in transactions:
        payer = transaction["payer"]
        amount = transaction["amount"]
        participants = transaction["participants"]
        split_amount = amount / len(participants)
        
        if payer not in balances:
            balances[payer] = 0
        balances[payer] += amount
        
        for participant in participants:
            if participant not in balances:
                balances[participant] = 0
            balances[participant] -= split_amount
    
    return balances


In [14]:
def calculate_individual_balances():
    balances = calculate_balances()
    debtors = {}
    creditors = {}

    for person, balance in balances.items():
        if balance < 0:
            debtors[person] = -balance
        elif balance > 0:
            creditors[person] = balance

    transactions = []

    for debtor, amount in debtors.items():
        for creditor, credit_amount in list(creditors.items()):
            if amount == 0:
                break

            payment = min(amount, credit_amount)
            transactions.append((debtor, creditor, payment))
            amount -= payment
            creditors[creditor] -= payment
            if creditors[creditor] == 0:
                del creditors[creditor]

    return transactions


In [15]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

class SplitwiseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Splitwise-like App")

        # Add transaction form
        self.payer_label = tk.Label(root, text="Payer")
        self.payer_label.grid(row=0, column=0)
        self.payer_entry = tk.Entry(root)
        self.payer_entry.grid(row=0, column=1)

        self.amount_label = tk.Label(root, text="Amount")
        self.amount_label.grid(row=1, column=0)
        self.amount_entry = tk.Entry(root)
        self.amount_entry.grid(row=1, column=1)

        self.desc_label = tk.Label(root, text="Description")
        self.desc_label.grid(row=2, column=0)
        self.desc_entry = tk.Entry(root)
        self.desc_entry.grid(row=2, column=1)

        self.participants_label = tk.Label(root, text="Participants (comma-separated)")
        self.participants_label.grid(row=3, column=0)
        self.participants_entry = tk.Entry(root)
        self.participants_entry.grid(row=3, column=1)

        self.add_button = tk.Button(root, text="Add Transaction", command=self.add_transaction)
        self.add_button.grid(row=4, column=0, columnspan=2)

        # Transactions list
        self.tree = ttk.Treeview(root, columns=("payer", "amount", "description", "participants"), show='headings')
        self.tree.heading("payer", text="Payer")
        self.tree.heading("amount", text="Amount")
        self.tree.heading("description", text="Description")
        self.tree.heading("participants", text="Participants")
        self.tree.grid(row=5, column=0, columnspan=2)

        # Balances label
        self.balances_label = tk.Label(root, text="Balances")
        self.balances_label.grid(row=6, column=0, columnspan=2)

        # Detailed balance label
        self.detailed_balances_label = tk.Label(root, text="Detailed Balances")
        self.detailed_balances_label.grid(row=7, column=0, columnspan=2)

        self.load_transactions()

        # Buttons for editing and deleting
        self.edit_button = tk.Button(root, text="Edit Transaction", command=self.edit_transaction)
        self.edit_button.grid(row=8, column=0)
        self.delete_button = tk.Button(root, text="Delete Transaction", command=self.delete_transaction)
        self.delete_button.grid(row=8, column=1)

    def add_transaction(self):
        payer = self.payer_entry.get()
        amount = float(self.amount_entry.get())
        description = self.desc_entry.get()
        participants = self.participants_entry.get().split(',')
        add_transaction(payer, amount, description, participants)
        self.load_transactions()
        messagebox.showinfo("Success", "Transaction added successfully")

    def edit_transaction(self):
        selected_item = self.tree.selection()[0]
        index = self.tree.index(selected_item)
        payer = self.payer_entry.get()
        amount = float(self.amount_entry.get())
        description = self.desc_entry.get()
        participants = self.participants_entry.get().split(',')
        edit_transaction(index, payer, amount, description, participants)
        self.load_transactions()
        messagebox.showinfo("Success", "Transaction edited successfully")

    def delete_transaction(self):
        selected_item = self.tree.selection()[0]
        index = self.tree.index(selected_item)
        delete_transaction(index)
        self.load_transactions()
        messagebox.showinfo("Success", "Transaction deleted successfully")

    def load_transactions(self):
        for row in self.tree.get_children():
            self.tree.delete(row)
        for transaction in transactions:
            self.tree.insert("", "end", values=(transaction["payer"], transaction["amount"], transaction["description"], ','.join(transaction["participants"])))
        self.update_balances()

    def update_balances(self):
        balances = calculate_balances()
        balances_text = "\n".join([f"{person}: ${amount:.2f}" for person, amount in balances.items()])
        self.balances_label.config(text=f"Balances:\n{balances_text}")

        individual_balances = calculate_individual_balances()
        detailed_balances_text = "\n".join([f"{debtor} owes {creditor}: ${amount:.2f}" for debtor, creditor, amount in individual_balances])
        self.detailed_balances_label.config(text=f"Detailed Balances:\n{detailed_balances_text}")

if __name__ == "__main__":
    root = tk.Tk()
    app = SplitwiseApp(root)
    root.mainloop()
